In [2]:
import tensorflow as tf;
print(tf.__version__)

2.10.0


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.10.0
GPU Available: []


In [4]:
from tensorflow.keras import mixed_precision
# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled")


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Mixed precision enabled


1.dataset loading and  normalization

In [5]:
#Set Paths and Basic Config

# Dataset paths
train_dir = "Train"
test_dir = "Test"

# Basic params
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123


In [6]:
# Create Training and Validation Datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.15,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.15,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 1748 files belonging to 7 classes.
Using 1486 files for training.
Found 1748 files belonging to 7 classes.
Using 262 files for validation.


In [7]:
# Create Test Dataset

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 697 files belonging to 7 classes.


In [8]:
#Normalize and Optimize


normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y),
                         num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y),
                     num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y),
                       num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


2.Data Augmentation
(dataset is fixed in size. Augmentation artificially increases variation (rotations, flips, zoom) to make the model generalize better and avoid overfitting)

In [9]:
from tensorflow.keras import layers

# Data Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # Flip images horizontally
    layers.RandomRotation(0.1),       # Rotate by ±10%
    layers.RandomZoom(0.1),           # Zoom by ±10%
])


In [10]:
import matplotlib.pyplot as plt
import numpy as np

# Strong augmentation for demonstration
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomContrast(0.5),
    tf.keras.layers.RandomTranslation(0.2, 0.2)
])

# Take one batch
for images, labels in train_ds.take(1):
    original_image = images[0].numpy().astype(np.float32)
    augmented_images = [data_augmentation(images)[0].numpy().astype(np.float32) for _ in range(5)]

# Plot original and augmented
plt.figure(figsize=(12, 6))
plt.subplot(1, 6, 1)
plt.imshow(original_image)
plt.title("Original")
plt.axis('off')

for i in range(5):
    plt.subplot(1, 6, i+2)
    plt.imshow(augmented_images[i])
    plt.title(f"Aug {i+1}")
    plt.axis('off')

plt.show()


InvalidArgumentError: Exception encountered when calling layer "random_contrast" "                 f"(type RandomContrast).

No OpKernel was registered to support Op 'AdjustContrastv2' used by {{node loop_body/adjust_contrast}} with these attrs: [T=DT_HALF]
Registered devices: [CPU]
Registered kernels:
  device='CPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_HALF]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_HALF]
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_HALF]

	 [[loop_body/adjust_contrast]] [Op:__inference_f_6076]

Call arguments received by layer "random_contrast" "                 f"(type RandomContrast):
  • inputs=tf.Tensor(shape=(32, 224, 224, 3), dtype=float16)
  • training=True

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# 1️⃣ Load EfficientNetB0 base (pre-trained on ImageNet)
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,          # we’ll add our own classifier
    input_shape=(224, 224, 3)   # same as your dataset images
)

base_model.trainable = False   # freeze feature extractor for now

# 2️⃣ Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(7, activation='softmax')(x)  # 7 = number of fish classes

model = Model(inputs=base_model.input, outputs=outputs)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling_1[0][0]']            
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      